# Rule curve - optimising the reservoir operating policy
## Reservoir system

<left><img src="Images/System 1 layout.png" width = "500px"><left>
## 0. Import libraries
To run this Notebook we need to import some necessary libraries. **Only if iRONs is run locally**: since one required library, [plotly](https://plot.ly/), is not available on Anaconda by default, you must have installed it first. Help on how to install libraries is given here: [How to install libraries](../0%20-%20Tutorials/0.b%20-%20How%20to%20install%20libraries.ipynb). If iRONs is run on the cloud, e.g. on [Binder](https://mybinder.org/) or [Microsoft Azure Notebooks](https://notebooks.azure.com/), we do not need to install the libraries to import them. 

Once all the necessary libraries are installed locally or in case we are running iRONs on the cloud, we import the libraries with the following code:

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from ipywidgets import widgets
import sys
# Submodules
sys.path.append('../../Submodules')
from Operation_rules import release_rule
from clim_dem_forecast import forecast

## 1. Release policy
In this notebook we will generate a rule curve for the regulated releases of the reservoir. Weekly release is scaled as a fraction of the long-term (five years or more) mean release (u_mean) while storage is scaled by the reservoir active capacity (0 at dead storage and 1 for full storage)
### 1.1 Parameters
Parameters that are going to define the coordinates of the points delineating the rule curve. u_min and u_max at minimum and maximum storage, u_mean is the long-term mean release and the coordinates u_ref and s_ref_1 and s_ref_2 of the (reference) inflection points.

In [2]:
# Input parameters
u_0 = 0.2 # release fraction
u_1 = 5 # release fraction

### 1.2 Rule curve for regulated reservoir releases
The rule curve is delineated by 4 points (x0, x1, x2 and x3) which are defined by the parameters set in the cell above.

Plotting the reservoir rule for regulated releases

## 1. Inputs

In [6]:
N = 8 # weeks
I = np.random.normal(loc=20,scale=5,size=N)
e = np.random.normal(loc=4,scale=1,size=N)
s_0 = 40 # ML
s_min = 0 # ML
s_max = 100 # ML

u_mean = 20

Qreq_env = 2
d = 25

## 2. Simulation

In [7]:
from Res_sys_sim import Res_sys_sim

In [8]:
def update_operating_policy(s_ref_1,s_ref_2,u_ref):
    x0 = [0,       u_0]
    x1 = [s_ref_1, u_ref]
    x2 = [s_ref_2, u_ref]
    x3 = [1,       u_1]
    param = [x0, x1, x2, x3, u_mean]
    u_frac = release_rule(param)/u_mean
    reg_releases = {'file_name' : 'Operation_rules',
                    'function' : 'release_rule',
                    'param': param}

    reg_inflows = None

    reg_rel_inf = None

    Qreg = {'releases' : reg_releases,
            'inflows'  : reg_inflows,
            'rel_inf'  : reg_rel_inf}
    Qenv, Qspill, u, I_reg, s = Res_sys_sim(I, e, s_0, s_min, s_max, Qreq_env, d, Qreg)
    return u_frac, Qenv, Qspill, u, I_reg, s

# Function to update the figure when changing the parameters with the sliders
def update_figure(change):
    with fig_1.batch_animate(duration=1000):
        fig_1.data[0].x = x
        fig_1.data[0].y = update_operating_policy(s_ref_1.value,s_ref_2.value,u_ref.value)[0]
        fig_1.layout.xaxis.title = 'Storage fraction'
        fig_1.layout.yaxis.title = 'Release fraction of long term flow'
        fig_1.layout.title = 'Operating policy for regulated releases'
    with fig_2.batch_animate(duration=1000):
        fig_2.data[0].x = np.arange(0,9)
        fig_2.data[0].y = update_operating_policy(s_ref_1.value,s_ref_2.value,u_ref.value)[5]
        fig_2.layout.xaxis.title = 'week'
        fig_2.layout.yaxis.title = 'Storage (ML)'
        fig_2.layout.title = 'Operating policy for regulated releases'        

x = np.arange(0,1.01,0.01)
# Definition of the sliders
u_ref = widgets.FloatSlider(min=0.5,
                            max=5,
                            value=1,
                            step=0.05,
                            description = 'Ref release: ',
                            continuous_update=False)
u_ref.observe(update_figure,names = 'value')
s_ref_1 = widgets.FloatSlider(min=0,
                              max=0.5,
                              value=0.25, 
                              step=0.05, 
                              description = 's_ref_1: ',
                              continuous_update=False)
s_ref_1.observe(update_figure,names = 'value')
s_ref_2 = widgets.FloatSlider(min=0.5,
                              max=1,
                              value=0.75, 
                              step=0.05,
                              description = 's_ref_2: ',
                              continuous_update=False)
s_ref_2.observe(update_figure,names = 'value')
### Figure with one trace: release rule ###
x0 = [0,             u_0]
x1 = [s_ref_1.value, u_ref.value]
x2 = [s_ref_2.value, u_ref.value]
x3 = [1,             u_1]
param = [x0, x1, x2, x3, u_mean]
trace_1 = go.Scatter(x = x,
                     y = update_operating_policy(s_ref_1.value,s_ref_2.value,u_ref.value)[0],
                     name='Sine curve')
trace_2 = go.Scatter(x = np.arange(0,10),
                     y = update_operating_policy(s_ref_1.value,s_ref_2.value,u_ref.value)[5])
fig_1 = go.FigureWidget(data=[trace_1],
                    layout=go.Layout(
                        xaxis = dict(
                            range = [0,1],
                            title = 'Storage fraction'),
                        yaxis = dict(
                            range = [0,5],
                            title = 'Release fraction of long term flow'),
                        )
                     )
fig_2 = go.FigureWidget(data=[trace_2],
                    layout=go.Layout(
                        xaxis = dict(
                            range = [0,8],
                            title = 'Week'),
                        yaxis = dict(
                            range = [0,110],
                            title = 'Storage (ML)'),
                        )
                     )
widgets.HBox([widgets.VBox([u_ref,s_ref_1,s_ref_2,fig_1]),
              fig_2])       

In [ ]:
update_operating_policy(s_ref_1.value,s_ref_2.value,u_ref.value)[3]

## 3. Optimization of the reservoir release rule

In [ ]:
# Optimizer
from platypus import NSGAII, Problem, Real, Integer

def auto_optim(vars):

    u_ref   = vars[0]
    s_ref_1 = vars[1]
    s_ref_2 = vars[2]
    
    x0 = [0,       u_min/u_mean]
    x1 = [s_ref_1, u_ref]
    x2 = [s_ref_2, u_ref]
    x3 = [1,       u_max/u_mean]

    param = [x0, x1, x2, x3, u_mean]
    
    Qreg = {'releases' : {'file_name' : 'Operation_rules',
                         'function' : 'release_rule',
                         'param': param},
            'inflows' : None,
            'rel_inf' : None}
    
    Qenv, Qspill, u, I_reg, s = Res_sys_sim(I,e,s_0,s_min,s_max,Qreq_env,Qreq_dem,Qreg)
    
    g_sup = np.maximum(Qreq_dem-u,0) # supply deficit objective
    g_res = s_max-s # resource availability objective
    J_sup = np.mean(g_sup[1:])
    J_res = np.mean(g_res[1:])
    
    constraints = [s_ref_2-s_ref_1]
    
    return [J_sup, J_res], constraints

problem = Problem(3,2,1)
Real0 = Real(0,u_max); Real1 = Real(0, 1); Real2 = Real(0, 1)

problem.types[:] = [Real0] + [Real1] + [Real2]
problem.constraints[:] = ">=0"
problem.function = auto_optim

population_size = 30
algorithm = NSGAII(problem,population_size)
algorithm.run(1000) # Number of iterations

results1_optim = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

sol_optim = [algorithm.result[i].variables for i in range(population_size)]

### 3.1 Results

In [ ]:
# First we define the traces (layers of data)
trace_4a = go.Scatter(x       = results1_optim,
                      y       = results2_optim,
                      name    = None,
                      mode    = 'markers',
                      marker  = dict(color='black'))
# Then the figure layout
layout_4 = go.Layout(title  = 'Tradeoff between objectives',
                     xaxis  = dict(showgrid=False,title = 'supply defict [ML]'),
                     yaxis  = dict(title = 'volume to maximum capacity [ML]'),
                     width  = 500, 
                     height = 500)
# Finally we plot figure using the pre-defined layout and traces
fig_4 = go.Figure(data=[trace_4a],
                  layout = layout_4)
fig_4

In [ ]:
x = np.arange(0,1.01,0.01)
layout_5 = go.Layout(title  = "Reservoir rule for regulated releases",
                     xaxis  = dict(showgrid=False,title = 'Storage fraction'),
                     yaxis  = dict(title = 'Release fraction of long term flow'),
                     width  = 600, 
                     height = 500,
                    showlegend = False)
fig_5 = go.Figure(layout = layout_5)
# First we define the traces (layers of data)

for i in range(population_size):
    fig_5.add_trace(go.Scatter(x = x,
                          y       = release_rule([[0,               u_min/u_mean],
                                                  [sol_optim[i][1], sol_optim[i][0]],
                                                  [sol_optim[i][2], sol_optim[i][0]],
                                                  [1,               u_max/u_mean],
                                                  u_mean]),
                          name    = 'rule curve '+str(i), 
                          line    = dict(color='blue', width=2)))
fig_5